<a href="https://colab.research.google.com/github/Hemalatha-mahesh/imdb-2024-data-scrapping/blob/main/data_insertion_sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install pandas

In [2]:
! pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 31.3 MB/s eta 0:00:00


In [3]:
import pandas as pd
import mysql.connector

# === Step 1: Read CSV file ===
csv_file = "/content/cleaned_data.csv"   # Change this to your CSV file path
df = pd.read_csv(csv_file)

# Handle missing values (important for SQL insert)
df = df.fillna(0)

# === Step 2: Connect to TiDB Cloud ===
mydb = mysql.connector.connect(
    host="gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
    port=4000,
    user="3ZnYqfQTEsJNQce.root",
    password="gybKhZEBwPo9Zwxr",
    database="testdb"   # 👈 change to your TiDB database name
)
print("Connected:", mydb.is_connected())

mycursor = mydb.cursor(buffered=True)

# === Step 3: Create table if not exists ===
mycursor.execute("""
CREATE TABLE IF NOT EXISTS movies (
    Movie_Name VARCHAR(255),
    Genre VARCHAR(100),
    Duration INT,
    Ratings FLOAT,
    Voting_counts INT
)
""")

# === Step 4: Prepare bulk insert ===
sql = """INSERT INTO movies (Movie_Name, Genre, Duration, Ratings, Voting_counts)
         VALUES (%s, %s, %s, %s, %s)"""

values = df[['Movie_Name', 'Genre', 'Duration', 'Ratings', 'Voting_counts']].values.tolist()

# === Step 5: Execute bulk insert ===
mycursor.executemany(sql, values)
mydb.commit()

print(f"Inserted {mycursor.rowcount} rows into TiDB Cloud")

# === Step 6: Close connection ===
mycursor.close()
mydb.close()


Connected: True
Inserted 1458 rows into TiDB Cloud
